In [1]:
import os
import json

In [18]:
file_path = os.path.join("..", "data", "results_github.json")
data = json.load(open(file_path))

In [19]:
# All github repositories, which at least contain one module susceptible to a gadget chain
cnt = 0
repo_names = []

for project in data:
    repo_name = "-".join(project.split("-")[1:])
    
    has_vuln_module = False
    for module in data[project]:
        if "payloads" in data[project][module]:
            has_vuln_module = True
            break
    
    if has_vuln_module:
        cnt += 1
        repo_names.append(project)
      
print(repo_names)  
print(f"{cnt} repositories contain at least one module with dependencies enabling a gadget")
    

['OpenRefine-OpenRefine', 'apache-systemds', 'oracle-tribuo', 'alibaba-velocity-spring-boot-project', 'apache-camel', 'jenkinsci-slack-plugin', 'jenkinsci-qy-wechat-notification-plugin', 'jenkinsci-docker-slaves-plugin', 'apache-hadoop', 'apache-phoenix', 'zendesk-maxwell', 'alibaba-spring-cloud-alibaba', 'jenkinsci-git-parameter-plugin', 'keycloak-keycloak', 'jenkinsci-jacoco-plugin', 'apache-skywalking', 'apache-cloudstack', 'jenkinsci-kubernetes-plugin', 'dromara-dynamic-tp', 'jenkinsci-gitlab-plugin', 'apache-avro', 'apache-shenyu', 'jenkinsci-docker-build-publish-plugin', 'jenkinsci-role-strategy-plugin', 'jenkinsci-generic-webhook-trigger-plugin', 'apache-activemq', 'jenkinsci-email-ext-plugin', 'jenkinsci-docker-workflow-plugin', 'apache-flume', 'apache-atlas', 'jenkinsci-subversion-plugin', 'apache-incubator-streampark', 'apache-kylin', 'jenkinsci-stashnotifier-plugin', 'alibaba-Appactive', 'eclipse-kapua', 'apache-mahout', 'apache-activemq-artemis', 'elastic-support-diagnostic

In [20]:
# Which payloads can be found / are most frequent

payload_cnts = {}

for project in data:
    
    for module in data[project]:
        if "payloads" in data[project][module]:
            for payload in data[project][module]["payloads"]:
                if not payload in payload_cnts:
                    payload_cnts[payload] = 0
                
                payload_cnts[payload] += 1
                
payload_cnts = sorted(payload_cnts.items(), key=lambda item: item[1], reverse=True)
print(payload_cnts)


[('CommonsBeanutils2', 1112), ('CommonsBeanutils1', 1104), ('AspectJWeaver', 360), ('CommonsCollections5', 92), ('CommonsCollections3', 92), ('CommonsCollections6', 92), ('CommonsCollections7', 92), ('CommonsCollections1', 92), ('SpringJta', 58), ('Click1', 36), ('CreateZeroFile', 34), ('Ssrf', 34), ('C3P0', 33), ('Jython4', 10), ('Jython3', 9), ('FileUpload1', 9), ('Lazylist', 4), ('ROME', 1), ('BeanShell1', 1), ('ROME2', 1)]


In [44]:
# Which dependencies enable payloads the most 

class Dependency:
    def __init__(self, name):
        self.name = name
        self.versions = {}
        self.total = 0
        self.projects_total = []
    
    def add_finding(self, version, project):
        self.total += 1
        if not version in self.versions:            
            self.versions[version] = 0
        self.versions[version] += 1
        
        if not project in self.projects_total:
            self.projects_total.append(project)
        
    def __str__(self):
        sorted_versions = sorted(self.versions.items(), key=lambda item: item[1], reverse=True)
        return str(self.total) + " & " + str(len(self.projects_total)) + " & " + str(sorted_versions).replace("[", "").replace("]", "")

dependency_cnts = {}

for project in data:
    
    for module in data[project]:
        for dep_obj in data[project][module]["vuln_dependencies"]:    
            if "enables_payload" in dep_obj:
                
                d_name = dep_obj["dependency"].replace("javax.", "")
                if not d_name in dependency_cnts:                    
                    dependency_cnts[d_name] = Dependency(d_name)
                dependency_cnts[d_name].add_finding(dep_obj["version"], project)

dependency_cnts = sorted(dependency_cnts.items(), key=lambda item: item[1].total, reverse=True)

print(len(dependency_cnts))
for dep in dependency_cnts:
    print("\\textbf{" + dep[0] + "} & " + str(dep[1]) + " \\\\")
    


20
\textbf{commons-collections} & 1180 & 137 & ('3.2.2', 1088), ('3.2.1', 92) \\
\textbf{commons-logging} & 1118 & 127 & ('1.2', 698), ('1.1.1', 211), ('1.1.3', 204), ('1.0.3', 3), ('1.0.4', 2) \\
\textbf{commons-beanutils} & 1112 & 123 & ('1.9.4', 960), ('1.9.2', 127), ('1.9.3', 25) \\
\textbf{aspectjweaver} & 360 & 22 & ('1.8.13', 127), ('1.9.7', 119), ('1.9.6', 65), ('1.8.9', 15), ('1.8.10', 8), ('1.9.21', 8), ('1.9.20.1', 5), ('1.9.0', 4), ('1.9.20', 3), ('1.8.6', 2), ('1.9.5', 2), ('1.9.9.1', 2) \\
\textbf{servlet-api} & 72 & 10 & ('2.5', 32), ('3.1.0', 21), ('4.0.1', 15), ('2.4', 4) \\
\textbf{spring-context} & 58 & 14 & ('5.3.22', 19), ('5.2.23.RELEASE', 16), ('5.3.30', 4), ('5.3.27', 4), ('4.3.9.RELEASE', 4), ('5.2.21.RELEASE', 3), ('5.3.31', 2), ('4.3.20.RELEASE', 1), ('4.3.30.RELEASE', 1), ('4.3.7.RELEASE', 1), ('5.3.20', 1), ('5.1.1.RELEASE', 1), ('5.3.2', 1) \\
\textbf{spring-beans} & 57 & 14 & ('5.3.22', 19), ('5.2.23.RELEASE', 16), ('5.3.30', 4), ('4.3.9.RELEASE', 4), ('5